In [1]:
# Import the libraries
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, accuracy_score, classification_report, precision_recall_fscore_support
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [2]:
# datasets
df1 = pd.read_excel('./data/case_study1.xlsx')
df2 = pd.read_excel('./data/case_study2.xlsx')

# make a copy of dataset
dt1 = df1.copy()
dt2 = df2.copy()

# Missing values Imputation and Removal

In [3]:
# df1 has co-occurring missing values in Age_Oldest_TL and Age_Newest_TL
df1[df1['Age_Oldest_TL'] == -99999].shape[0]

40

In [4]:
# Drop the rows as it only has 41 missing rows
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999] # -99999 Represents the null values

Let's clean up df2 by getting rid of columns that have more than 10,000 missing values, which is over 20% of the data. It's important not to mess too much with the sensitive info, because if we try to fill in more than 20% of the missing stuff, it might mess up how everything is spread out.

In [5]:
col_to_drop = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        col_to_drop.append(i)

In [6]:
col_to_drop

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [7]:
df2.drop(columns=col_to_drop, inplace=True)

In [8]:
# Removing rows with null values
for i in df2.columns:
    df2 = df2.loc[df2[i] != -99999]

In [9]:
# Display the precision, recall, and F1 score for each class
# Display the precision, recall, and F1 score for each class
print(df1.shape, df2.shape)

(51296, 26) (42066, 54)


In [10]:
# Checking for any other null values
# Check for any missing values in the dataset
# Check for any missing values in the dataset
print(df1.isnull().sum()[df1.isnull().sum() != 0])
# Check for any missing values in the dataset
# Check for any missing values in the dataset
print(df2.isnull().sum()[df2.isnull().sum() != 0])

Series([], dtype: int64)
Series([], dtype: int64)


In [11]:
# Merge both data using key columns
for i in list(df1.columns):
    if i in list(df2.columns):
# Display the precision, recall, and F1 score for each class
# Display the precision, recall, and F1 score for each class
        print(i)

PROSPECTID


In [12]:
df = pd.merge(df1, df2, how='inner', on=['PROSPECTID'])

# Categorical Feature Preprocessing

In [13]:
# Check categorical columns
cat_features = []
for i in df.columns:
    if df[i].dtype == 'object':
        cat_features.append(i)
        print(df[i].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']
['P2' 'P1' 'P3' 'P4']


In [14]:
# Check if the categorical columns and Approved_Flag are correlated
for i in cat_features:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, ':', pval)

MARITALSTATUS : 3.578180861038862e-233
EDUCATION : 2.6942265249737532e-30
GENDER : 1.907936100186563e-05
last_prod_enq2 : 0.0
first_prod_enq2 : 7.84997610555419e-287
Approved_Flag : 0.0


Every feature shows a p-value below 0.05, meaning they're strongly linked to the output variable and probably crucial for making predictions.

# Numerical Feature Preprocessing

In [15]:
# Check the numerical columns
num_features = []
for i in df.columns:
    if df[i].dtype != 'object' and i != 'PROSPECTID':
        num_features.append(i)

In [16]:
num_features

['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

In [17]:
# Calculate the VIF to check for multicolinearity in the features
vif_data = df[num_features]
total_columns = vif_data.shape[1]
columns_to_keep = []
column_index = 0

In [18]:
for i in range(0, total_columns):
    vif_value = variance_inflation_factor(vif_data, column_index)
# Display the precision, recall, and F1 score for each class
# Display the precision, recall, and F1 score for each class
    print(column_index, ':', vif_value)
    
    if vif_value <= 6:
        columns_to_keep.append(num_features[i])
        column_index = column_index + 1
        
    else:
        vif_data = vif_data.drop([num_features[i]], axis=1)

C:\Users\atulk\AppData\Local\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 : inf


C:\Users\atulk\AppData\Local\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 : inf
0 : 11.320180023967996
0 : 8.363698035000327
0 : 6.520647877790928
0 : 5.149501618212625
1 : 2.611111040579735


C:\Users\atulk\AppData\Local\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 : inf
2 : 1788.7926256209232
2 : 8.601028256477228
2 : 3.8328007921530785
3 : 6.099653381646739
3 : 5.5813520096427585
4 : 1.985584353098778


C:\Users\atulk\AppData\Local\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 : inf
5 : 4.809538302819343
6 : 23.270628983464636
6 : 30.595522588100053
6 : 4.3843464059655854
7 : 3.0646584155234238
8 : 2.898639771299253
9 : 4.377876915347324
10 : 2.207853583695844
11 : 4.916914200506864
12 : 5.214702030064725
13 : 3.3861625024231476
14 : 7.840583309478997
14 : 5.255034641721438


C:\Users\atulk\AppData\Local\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 : inf
15 : 7.380634506427232
15 : 1.4210050015175733
16 : 8.083255010190316
16 : 1.6241227524040112
17 : 7.257811920140003
17 : 15.59624383268298
17 : 1.825857047132431
18 : 1.5080839450032664
19 : 2.172088834824578
20 : 2.6233975535272274
21 : 2.2959970812106167
22 : 7.360578319196446
22 : 2.1602387773102554
23 : 2.8686288267891467
24 : 6.458218003637277
24 : 2.8474118865638265
25 : 4.753198156284083
26 : 16.22735475594825
26 : 6.424377256363877
26 : 8.887080381808687
26 : 2.3804746142952653
27 : 8.60951347651454
27 : 13.06755093547673
27 : 3.500040056654654
28 : 1.9087955874813773
29 : 17.006562234161628
29 : 10.730485153719197
29 : 2.3538497522950275
30 : 22.104855915136433
30 : 2.7971639638512906
31 : 3.424171203217696
32 : 10.175021454450922
32 : 6.408710354561301
32 : 1.001151196262562
33 : 3.069197305397274
34 : 2.8091261600643724
35 : 20.249538381980678
35 : 15.864576541593774
35 : 1.8331649740532172
36 : 1.5680839909542037
37 : 1.9307572353811682
38 : 4.331265056645247
39 :

After using Variance Inflation Factor (VIF) analysis, we've narrowed down to just 39 features. We set the VIF threshold at 6 to make sure our model is easy to understand, and we want to get rid of any features that are basically saying the same thing as others.

In [19]:
# df_data = df.copy()

In [20]:
# min_max_data = []
# for i in df_data[num_features].columns:
#     min_val = min(df_data[i])
#     max_val = max(df_data[i])
#     min_max_data.append({'Column': i, 'Min Value': min_val, 'Max Value': max_val})
# min_max_df = pd.DataFrame(min_max_data)

In [21]:
# for i in df_data[num_features].columns:
#     if max(df_data[i]) <= 127:
#         if df[i].dtype not in ["float16", "float32", "float64"]:
#             df_data[i] = df_data[i].astype('int8')
#         else:
#             df_data[i] = df_data[i].astype('float16')
#     elif max(df_data[i]) <= 32767:
#         if df[i].dtype not in ["float16", "float32", "float64"]:
#             df_data[i] = df_data[i].astype('int16')
#         else:
#             df_data[i] = df_data[i].astype('float16')
#     elif max(df_data[i]) <= 2147483647:
#         if df[i].dtype not in ["float16", "float32", "float64"]:
#             df_data[i] = df_data[i].astype('int32')

Using anova test for feature selection

In [22]:
# Import necessary libraries
# Import necessary libraries
from scipy.stats import f_oneway

In [23]:
columns_to_be_kept_numerical = []

In [24]:
for i in columns_to_keep:
    a = list(df[i])
    b = list(df["Approved_Flag"])
    group_p1 = [value for value,group in zip(a,b) if group == "P1"]
    group_p2 = [value for value,group in zip(a,b) if group == "P2"]
    group_p3 = [value for value,group in zip(a,b) if group == "P3"]
    group_p4 = [value for value,group in zip(a,b) if group == "P4"]
    
    f_statistic,p_value = f_oneway(group_p1,group_p2,group_p3,group_p4)
    
    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

All the final features

In [25]:
features = columns_to_be_kept_numerical + cat_features
df = df[features]

In [26]:
# encoding
for i in cat_features:
    print(df[i].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']
['P2' 'P1' 'P3' 'P4']


In [27]:
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 4
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 5
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 0
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 6

In [28]:
df['EDUCATION'].value_counts()
df['EDUCATION'] = df['EDUCATION'].astype(int)

In [29]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'],dtype="int8")

In [30]:
df_encoded

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.00,0,0.000,0,0,0,4,1,4,...,0,0,1,0,0,0,0,0,1,0
1,0.000,0.00,0,0.000,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0.125,0.00,0,0.000,1,0,0,0,2,6,...,1,0,0,0,0,0,0,0,0,1
3,0.000,0.00,0,0.000,0,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
4,0.000,0.00,1,0.167,0,0,0,0,6,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,0.333,0.00,0,0.000,0,0,0,0,0,3,...,1,0,0,0,0,0,1,0,0,0
42060,0.000,0.25,1,0.250,0,0,0,0,2,2,...,0,0,0,1,0,0,0,0,0,1
42061,0.500,0.50,1,0.500,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,1
42062,0.000,0.00,1,0.500,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,1


In [31]:
y = df_encoded["Approved_Flag"]
X = df_encoded.drop(["Approved_Flag"],axis=1)

In [32]:
# Split the data into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [33]:
# Initialize the RandomForestClassifier with 200 trees
rf_classifier = RandomForestClassifier(n_estimators=200,random_state=42)

In [34]:
# Train the RandomForest model
rf_classifier.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [35]:
y_pred = rf_classifier.predict(X_test)

In [36]:
# Display the accuracy
accuracy = accuracy_score(y_test,y_pred)

In [37]:
accuracy

0.7650065375014858

In [38]:
precision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred)

In [39]:
for i,v in enumerate(["p1","p2","p3","p4"]):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")            

Class p1:
Precision: 0.8416075650118203
Recall: 0.7021696252465484
F1 Score: 0.7655913978494624
Class p2:
Precision: 0.7959390862944162
Recall: 0.932408325074331
F1 Score: 0.8587859424920127
Class p3:
Precision: 0.4472843450479233
Recall: 0.21132075471698114
F1 Score: 0.2870322911327524
Class p4:
Precision: 0.7177497575169738
Recall: 0.7191448007774538
F1 Score: 0.7184466019417475


Using Random Forest, we are achieving an accuracy score of 70%.

In [40]:
# XGboost 
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [41]:
xgb_classifier = xgb.XGBClassifier(objective='multi.softmax',num_class=4)

In [42]:
y = df_encoded["Approved_Flag"]
X = df_encoded.drop(["Approved_Flag"],axis=1)

In [43]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [44]:
# Split the data into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y_encoded,test_size=0.2,random_state=42)

In [45]:
xgb_classifier.fit(X_train,y_train)
y_pred  = xgb_classifier.predict(X_test)

In [46]:
accuracy = accuracy_score(y_test,y_pred)

In [47]:
accuracy

0.7733269939379531

In [48]:
precision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred)

In [49]:
for i,v in enumerate(["p1","p2","p3","p4"]):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")

Class p1:
Precision: 0.8300865800865801
Recall: 0.7564102564102564
F1 Score: 0.7915376676986583
Class p2:
Precision: 0.8212310437109723
Recall: 0.912388503468781
F1 Score: 0.8644131455399061
Class p3:
Precision: 0.4515752625437573
Recall: 0.2920754716981132
F1 Score: 0.35472043996333635
Class p4:
Precision: 0.7293086660175268
Recall: 0.7278911564625851
F1 Score: 0.7285992217898833


The accuracy score of XGBoost is 77%.

If the data is heavily imbalanced, use recall and precision. If the data is balanced, use the accuracy score.

Hyperparameter Tuning  for XGboost 

In [50]:
from sklearn.model_selection import GridSearchCV
X_train,X_test,y_train,y_test = train_test_split(X,y_encoded,test_size=0.2,random_state=42)

xgb_model = xgb.XGBClassifier(objective="multi:softmax",num_class = 4)

param_grid = {
    'colsample_bytree': [0.3, 0.6, 1],
    'learning_rate': [0.001, 0.01, 0.1],
    'max_depth': [3, 5, 10],
    'n_estimators': [50, 100, 200]
}

grid_search = GridSearchCV(estimator=xgb_model,param_grid=param_grid,cv=3,scoring="accuracy",n_jobs=-1)
grid_search.fit(X_train,y_train)
print(f"Best hyperparameters: {grid_search.best_params_}")

Best hyperparameters: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


<b> Best hyperparameters </b>: {'colsample_bytree': 0.9, 'learning_rate': 1, 'max_depth': 3,"alpha":10 'n_estimators': 100}

In [51]:
best_model = grid_search.best_estimator_
accuracy = best_model.score(X_test,y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.7786758587899679


In [52]:
Unseen_Dataset = pd.read_excel("data/Unseen_Dataset.xlsx")

In [53]:
Unseen_Dataset.loc[Unseen_Dataset['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
Unseen_Dataset.loc[Unseen_Dataset['EDUCATION'] == '12TH',['EDUCATION']]             = 2
Unseen_Dataset.loc[Unseen_Dataset['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 4
Unseen_Dataset.loc[Unseen_Dataset['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
Unseen_Dataset.loc[Unseen_Dataset['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 5
Unseen_Dataset.loc[Unseen_Dataset['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 0
Unseen_Dataset.loc[Unseen_Dataset['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 6

In [54]:
Unseen_Dataset['EDUCATION'] = Unseen_Dataset['EDUCATION'].astype(int)

In [55]:
df_encoded = pd.get_dummies(Unseen_Dataset, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'],dtype="int8")

In [56]:
df_encoded.shape

(100, 54)

In [57]:
Unseen_Dataset_result =pd.DataFrame(best_model.predict(df_encoded))

In [58]:
Unseen_Dataset_result.value_counts()

0
1    72
3    12
2     9
0     7
Name: count, dtype: int64

In [60]:
# import pickle

# # Assuming your model is named 'model'
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

NameError: name 'model' is not defined